In [1]:
import pandas as pd
import numpy as np  

In [2]:
train = pd.read_csv("./데이터/버스 운행시간 데이터/train.csv")
test = pd.read_csv("./데이터/버스 운행시간 데이터/test.csv")
submission = pd.read_csv("./데이터/버스 운행시간 데이터/submission_제출양식.csv")

id =해당 데이터에서의 고유한 ID(train, test와의 중복은 없음.)\
date=버스 운행 날짜\
route_id = 버스 노선 ID\
vh_id = 버스 ID\
route_nm = 버스 노선 실제 번호\
now_latitude = 현재 정류소의 위도\
now_longitude = 현재 정류소의 경도\
now_station = 현재 정류소 이름\
now_arrive_time = 현재 정류장에 도착한 시간\
distance = 현재 정류장에서 다음 정류장까지 실제 이동한 거리\
next_station = 다음 정류소 이름\
next_latitude = 다음 정류소의 위도\
next_longitude = 다음 정류소의 경도\
next_arrive_time = 다음 정류장에 도착할 때 까지 걸린 시간(단위:초)으로 답안 제출을 위해서 예측해야 되는 값\

In [3]:
train

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21시,528.0,삼아아파트,33.251896,126.574417,96
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21시,280.0,매일올레시장 7번입구,33.249753,126.565959,50
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21시,114.0,아랑조을거리 입구,33.251084,126.559551,16
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21시,223.0,평생학습관,33.249504,126.558068,38


In [4]:
test

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91769,302350,2019-11-05,405328102,7983486,281-2,33.493625,126.534764,제주지방법원(광양방면),20시,272.0,고산동산(광양방면),33.495540,126.532907
91770,302351,2019-11-05,405328102,7983486,281-2,33.495540,126.532907,고산동산(광양방면),20시,447.0,제주시청(광양방면),33.498925,126.530351
91771,302352,2019-11-05,405328102,7983486,281-2,33.498925,126.530351,제주시청(광양방면),20시,418.0,광양사거리,33.500473,126.527103
91772,302353,2019-11-05,405328102,7983486,281-2,33.500473,126.527103,광양사거리,20시,140.0,탐라장애인 종합복지관,33.500228,126.525625


In [33]:
feature = train.drop("next_arrive_time",axis=1)
target = train["next_arrive_time"]

In [34]:
feature 

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21시,528.0,삼아아파트,33.251896,126.574417
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21시,280.0,매일올레시장 7번입구,33.249753,126.565959
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21시,114.0,아랑조을거리 입구,33.251084,126.559551
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21시,223.0,평생학습관,33.249504,126.558068


In [35]:
feature["vh_id"].unique()

array([7997025, 7997034, 7997035, 7997040, 7997044, 7997058, 7997064,
       7997080, 7997118, 7997124, 7997043, 7997082, 7997111, 7997112,
       7997067, 7997028, 7997033, 7997001, 7997031, 7997070, 7997077,
       7997087, 7997089, 7997002, 7997022, 7997032, 7997056, 7997062,
       7997085, 7997086, 7997090, 7997095, 7997107, 7997110, 7997115,
       7997119, 7997120, 7983010, 7983041, 7983097, 7983005, 7983063,
       7983066, 7983095, 7983224, 7983075, 7983031, 7983038, 7983074,
       7983228, 7983096, 7983032, 7983036, 7983065, 7983073, 7983093,
       7983019, 7983227, 7983201, 7983092, 7983028, 7983051, 7983200,
       7983000, 7983013, 7983222, 7983026, 7983094, 7983014, 7983045,
       7983202, 7983226, 7983229, 7983402, 7983403, 7983416, 7983417,
       7983420, 7983431, 7983441, 7983481, 7983482, 7983483, 7983484,
       7983400, 7983406, 7983409, 7983412, 7983414, 7983415, 7983430,
       7983432, 7983439, 7983444, 7983445, 7983485, 7983486, 7997123,
       7997041, 7997

In [36]:
feature["route_nm"].unique()

array(['360-1', '360-2', '360-7', '360-12', '365-21', '365-22', '201-11',
       '201-12', '201-13', '201-14', '201-15', '201-16', '201-17',
       '201-18', '201-21', '201-22', '201-24', '201-26', '201-27',
       '281-1', '281-2'], dtype=object)

In [37]:
feature2 = feature.drop(["now_station", "next_station","id","date","route_nm","now_arrive_time"],axis=1)
feature2

,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude
0,405136001,7997025,33.456267,126.551750,266.0,33.457724,126.554014
1,405136001,7997025,33.457724,126.554014,333.0,33.458783,126.557353
2,405136001,7997025,33.458783,126.557353,415.0,33.459893,126.561624
3,405136001,7997025,33.479705,126.543811,578.0,33.484860,126.542928
4,405136001,7997025,33.485662,126.494923,374.0,33.485822,126.490897
...,...,...,...,...,...,...,...
210452,405328102,7983486,33.255783,126.577450,528.0,33.251896,126.574417
210453,405328102,7983486,33.248595,126.568527,280.0,33.249753,126.565959
210454,405328102,7983486,33.251891,126.560303,114.0,33.251084,126.559551
210455,405328102,7983486,33.251084,126.559551,223.0,33.249504,126.558068


In [38]:
test2 = test.drop(["now_station", "next_station","id","date","route_nm","now_arrive_time"],axis=1)
test2

,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude
0,405136001,7997025,33.457724,126.554014,333.0,33.458783,126.557353
1,405136001,7997025,33.458783,126.557353,415.0,33.459893,126.561624
2,405136001,7997025,33.478867,126.483833,417.0,33.478850,126.488350
3,405136001,7997025,33.478850,126.488350,334.0,33.480700,126.489933
4,405136001,7997025,33.480700,126.489933,550.0,33.482077,126.485355
...,...,...,...,...,...,...,...
91769,405328102,7983486,33.493625,126.534764,272.0,33.495540,126.532907
91770,405328102,7983486,33.495540,126.532907,447.0,33.498925,126.530351
91771,405328102,7983486,33.498925,126.530351,418.0,33.500473,126.527103
91772,405328102,7983486,33.500473,126.527103,140.0,33.500228,126.525625


In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [40]:
lr = LinearRegression()

In [50]:
import statsmodels.api as sm

feature_add = sm.add_constant(feature2,  has_constant='add')

# sm OLS 적합
model = sm.OLS(target , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       next_arrive_time   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     9400.
Date:                Mon, 06 Mar 2023   Prob (F-statistic):               0.00
Time:                        10:32:52   Log-Likelihood:            -1.2051e+06
No. Observations:              210457   AIC:                         2.410e+06
Df Residuals:                  210449   BIC:                         2.410e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -8.518e+04   2.18e+04     -3.914      0.000   -1.28e+05   -4.25e+04
route_id           0.0001    4.2e-05      3.468      0.001    6.34e-05       0.000
vh_id              0.0038      0.001      6.619      0.000       0.003       0.005
now_latitude    -283.5442     35.022     -8.096      0.000    -352.186    -214.903
now_longitude   -126.8180     43.135     -2.940      0.003    -211.361     -42.275
distance           0.0748      0.000    234.894      0.000       0.074       0.075
next_latitude    327.2641     35.048      9.338      0.000     258.572     395.957
next_longitude    81.6836     43.083      1.896      0.058      -2.758     166.125
==============================================================================
Omnibus:                   412603.939   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1623593765.589
Skew:                          15.568   Prob(JB):                         0.00
Kurtosis:                     432.163   Cond. No.                     5.45e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.45e+13. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
lr.fit(feature2, target)

LinearRegression()

In [47]:
lr.coef_

array([ 1.45717123e-04,  3.80936336e-03, -2.83544154e+02, -1.26818031e+02,
        7.47527807e-02,  3.27264122e+02,  8.16836211e+01])

In [42]:
pred = lr.predict(test2)
pred

array([ 92.66649251,  98.78463578, 102.78176251, ...,  77.92921735,
        56.92001483,  86.68421224])

In [43]:
submission 

,id,next_arrive_time
0,210457,0
1,210458,0
2,210459,0
3,210460,0
4,210461,0
...,...,...
91769,302350,0
91770,302351,0
91771,302352,0
91772,302353,0


In [45]:
submission2 = submission
submission2["next_arrive_time"] = pred

In [46]:
submission2

,id,next_arrive_time
0,210457,92.666493
1,210458,98.784636
2,210459,102.781763
3,210460,96.744009
4,210461,112.241995
...,...,...
91769,302350,66.818145
91770,302351,80.491402
91771,302352,77.929217
91772,302353,56.920015


In [49]:
submission2.to_csv("제출용.csv",index=False)

In [ ]:
# 데이터 전처리 1 날짜가 만약 평일인 경우 주말인 경우